In [1]:
!pip install --upgrade hifi-gan-bwe

In [2]:
!pip install scipy

In [3]:
import audioread
import numpy as np
import torch
from IPython.display import Audio
from hifi_gan_bwe import BandwidthExtender
import os

In [4]:
model = BandwidthExtender.from_pretrained("hifi-gan-bwe-10-42890e3-vctk-48kHz")

In [5]:
originalFilePath = "FullBandFiles/p232_001.wav"

In [6]:
import scipy.io.wavfile as wav
import matplotlib.pyplot as plt
from scipy.signal import decimate

fs, original_data = wav.read(originalFilePath)


In [7]:
new_fs = 16000
q = int(fs/new_fs)

def downsample_data(data):
    downsampled_data = decimate(data, q, ftype='fir', zero_phase=True) #Zero_phase=True hindrer faseforstyrrelse
    wav.write('downsampled_audio.wav', new_fs, downsampled_data.astype(np.int16))
    path = 'downsampled_audio.wav'

    return downsampled_data, path

#downsampled_data, filepath = downsample_data(original_data)

In [8]:
filepath = 'downsampled_audio.wav'
def preprocess_data(filepath):
    with audioread.audio_open(filepath) as input_:
        sample_rate = input_.samplerate
        x = (
            np.hstack([np.frombuffer(b, dtype=np.int16) for b in input_])
            .reshape([-1, input_.channels])
            .astype(np.float32)
            / 32767.0
        )
    return x
    
preprocessed_data = preprocess_data(filepath)

In [9]:
def run_model(preprocessed_data):
    with torch.no_grad():
        y = np.stack([model(torch.from_numpy(preprocessed_data), new_fs) for preprocessed_data in preprocessed_data.T]).T
    return y


    
y = run_model(preprocessed_data)





In [10]:
folderpath = 'FullBandFiles'
folderlist = os.listdir(folderpath)

index = 99

filepath_original = os.path.join(folderpath, folderlist[index])
#if filename[-4:].lower() != '.wav':
        
fs, original_data = wav.read(filepath_original)

downsampled_data, filepath = downsample_data(original_data)

prepped_data = preprocess_data(filepath)
bwe_data = run_model(prepped_data)

Audio(original_data.T, rate=fs, autoplay=False)




In [11]:
Audio(bwe_data.T, rate=int(model.sample_rate), autoplay=False)